In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
ls drive/MyDrive/CaseStudy/

'AirBnB_Prediction_Challenge_Case Study and guidelines.docx'   dl_model.h5
 bow_vectorizer.sav                                            ml_model.sav
'case study.zip'                                               submission.csv
'case study.zip (Unzipped Files)'/                             test.xlsx
 Cleaned_Train.csv                                             train.csv
 cleaning_fee_label_encoder.sav


As Random Forest worked better, we are retraining it and predicting for original test data

# Train

In [33]:
working_directory = "drive/MyDrive/CaseStudy/"

In [34]:
import pickle

def save_model(model, filename="model.sav"):
  pickle.dump(model, open(filename, 'wb'))
 

In [35]:
import numpy as np
import pandas as pd

In [36]:
Train_Dataset = pd.read_csv(working_directory + 'train.csv')

In [37]:
Train_Dataset.shape

(49999, 30)

In [38]:
Train_Dataset.isnull().sum()

id                            0
property_type                 0
room_type                     0
amenities                     0
accommodates                  0
bathrooms                   146
bed_type                      0
cancellation_policy           0
cleaning_fee                  0
city                          0
description                   0
first_review              10711
host_has_profile_pic        127
host_identity_verified      127
host_response_rate        12338
host_since                  127
instant_bookable              0
last_review               10683
latitude                      0
longitude                     0
name                          0
neighbourhood              4617
number_of_reviews             0
review_scores_rating      11301
thumbnail_url              5595
zipcode                     640
bedrooms                     62
beds                         91
log_price                     0
                          49999
dtype: int64

In [39]:
Train_Dataset.columns

Index(['id', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city',
       'description', 'first_review', 'host_has_profile_pic',
       'host_identity_verified', 'host_response_rate', 'host_since',
       'instant_bookable', 'last_review', 'latitude', 'longitude', 'name',
       'neighbourhood', 'number_of_reviews', 'review_scores_rating',
       'thumbnail_url', 'zipcode', 'bedrooms', 'beds', 'log_price', ' '],
      dtype='object')

In [40]:
Train_Dataset.head()

,id,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,log_price,
0,6901257,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",6/18/2016,t,t,NaN,3/26/2012,f,7/18/2016,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0,5.010635,NaN
1,6304928,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,8/5/2017,t,f,100%,6/19/2017,t,9/23/2017,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0,5.129899,NaN
2,7919400,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,4/30/2017,t,t,100%,10/25/2016,t,9/14/2017,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0,4.976734,NaN
3,13418779,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,4/19/2015,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117,2.0,2.0,6.620073,NaN
4,3808709,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",5/12/2015,t,t,100%,3/1/2015,t,1/22/2017,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0,4.744932,NaN


In [41]:
Train_Dataset.dtypes

id                          int64
property_type              object
room_type                  object
amenities                  object
accommodates                int64
bathrooms                 float64
bed_type                   object
cancellation_policy        object
cleaning_fee                 bool
city                       object
description                object
first_review               object
host_has_profile_pic       object
host_identity_verified     object
host_response_rate         object
host_since                 object
instant_bookable           object
last_review                object
latitude                  float64
longitude                 float64
name                       object
neighbourhood              object
number_of_reviews           int64
review_scores_rating      float64
thumbnail_url              object
zipcode                    object
bedrooms                  float64
beds                      float64
log_price                 float64
              

In [42]:
cleaned_train_dataset = Train_Dataset.drop(['id',
                                            'zipcode',
                                            'cancellation_policy',
                                            'description','first_review',
                                            'host_has_profile_pic',
                                            'host_identity_verified',
                                            'host_response_rate',
                                            'host_since',
                                            'instant_bookable',
                                            'last_review', 
                                            'latitude', 
                                            'longitude', 
                                            'name',
                                            'neighbourhood',
                                            'thumbnail_url',
                                            ' '],axis = 'columns',inplace = False)

In [43]:
cleaned_train_dataset.head()

,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cleaning_fee,city,number_of_reviews,review_scores_rating,bedrooms,beds,log_price
0,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,True,NYC,2,100.0,1.0,1.0,5.010635
1,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,True,NYC,6,93.0,3.0,3.0,5.129899
2,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,True,NYC,10,92.0,1.0,3.0,4.976734
3,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,True,SF,0,NaN,2.0,2.0,6.620073
4,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,True,DC,4,40.0,0.0,1.0,4.744932


property_type: Categorial,
  room_type: Categorical,
  amenities: Categorical list,
  accommodates: numerical,
  bathrooms: numerical(float),
  bed_type: categorical,
  cleaning_fee: Boolean,
  review_scores_rating: numberical float,
  number_of_reviews: numerical,
  bedrooms: numerical,
  beds:numerical,
  log_price: numerical,

In [44]:
cleaned_train_dataset.shape

(49999, 13)

### Missing Values Handling

In [45]:
cleaned_train_dataset.isnull().sum()

property_type               0
room_type                   0
amenities                   0
accommodates                0
bathrooms                 146
bed_type                    0
cleaning_fee                0
city                        0
number_of_reviews           0
review_scores_rating    11301
bedrooms                   62
beds                       91
log_price                   0
dtype: int64

In [46]:
sum(cleaned_train_dataset.apply(lambda x: sum(x.isnull().values), axis = 1)>0)

11477

Above we can observe that summation of all missing values is 11477, review_score_rating has highest missing values among all.

In [47]:
# Replacing missing values with 0 for review_scores_rating column
cleaned_train_dataset['review_scores_rating'] = cleaned_train_dataset['review_scores_rating'].fillna(0)

In [48]:
total_missing_values = sum(cleaned_train_dataset.apply(lambda x: sum(x.isnull().values), axis = 1)>0)

In [49]:
# Percentage of missing values in the given dataset
(total_missing_values/len(cleaned_train_dataset)) * 100

0.47800956019120383

As missing values are very low i.e., 0.47%, will do not have much affect in the results

In [50]:
cleaned_train_dataset.dropna(inplace=True)

In [51]:
cleaned_train_dataset.isnull().sum()

property_type           0
room_type               0
amenities               0
accommodates            0
bathrooms               0
bed_type                0
cleaning_fee            0
city                    0
number_of_reviews       0
review_scores_rating    0
bedrooms                0
beds                    0
log_price               0
dtype: int64

In [52]:
cleaned_train_dataset.dtypes

property_type            object
room_type                object
amenities                object
accommodates              int64
bathrooms               float64
bed_type                 object
cleaning_fee               bool
city                     object
number_of_reviews         int64
review_scores_rating    float64
bedrooms                float64
beds                    float64
log_price               float64
dtype: object

### Label Encoding

In [53]:
from sklearn.preprocessing import LabelEncoder

In [54]:
cleaning_fee_label_encoder = LabelEncoder()
cleaning_fee_label_encoder.fit(cleaned_train_dataset['cleaning_fee'])

LabelEncoder()

In [55]:
cleaning_fee_label_encoder.classes_

array([False,  True])

In [56]:
cleaned_train_dataset['cleaning_fee'].dtype

dtype('bool')

In [57]:
cleaned_train_dataset['cleaning_fee'] =  cleaning_fee_label_encoder.transform(cleaned_train_dataset['cleaning_fee'])

In [58]:
cleaned_train_dataset['cleaning_fee'].dtype

dtype('int64')

In [59]:
cleaned_train_dataset['cleaning_fee']

0        1
1        1
2        1
3        1
4        1
        ..
49994    0
49995    1
49996    1
49997    1
49998    0
Name: cleaning_fee, Length: 49760, dtype: int64

In [60]:
save_model(cleaning_fee_label_encoder, working_directory+"cleaning_fee_label_encoder.sav")

### Handling amenties feature

In [61]:
for i in cleaned_train_dataset['amenities'][:10]:
  print(i)

{"Wireless Internet","Air conditioning",Kitchen,Heating,"Family/kid friendly",Essentials,"Hair dryer",Iron,"translation missing: en.hosting_amenity_50"}
{"Wireless Internet","Air conditioning",Kitchen,Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Fire extinguisher",Essentials,Shampoo,Hangers,"Hair dryer",Iron,"translation missing: en.hosting_amenity_50"}
{TV,"Cable TV","Wireless Internet","Air conditioning",Kitchen,Breakfast,"Buzzer/wireless intercom",Heating,"Family/kid friendly","Smoke detector","Carbon monoxide detector","Fire extinguisher",Essentials,Shampoo,Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_50"}
{TV,"Cable TV",Internet,"Wireless Internet",Kitchen,"Indoor fireplace","Buzzer/wireless intercom",Heating,Washer,Dryer,"Smoke detector","Carbon monoxide detector","First aid kit","Fire extinguisher",Essentials}
{TV,Internet,"Wireless Internet","Air conditioning",Kitchen,"Elevator in building",Heating,"Smoke detecto

In [62]:
def process_amenities(text):
  words = text.replace("{","").replace("}","").replace('"',"").replace(' ',"").split(",")
  if "translation missing:en.hosting_amenity_50" in words:
    return " ".join(words[:-1])
  else:
    return " ".join(words)

In [63]:
cleaned_train_dataset['amenities'] = cleaned_train_dataset['amenities'].apply(process_amenities)

In [64]:
cleaned_train_dataset['amenities']

0        WirelessInternet Airconditioning Kitchen Heati...
1        WirelessInternet Airconditioning Kitchen Heati...
2        TV CableTV WirelessInternet Airconditioning Ki...
3        TV CableTV Internet WirelessInternet Kitchen I...
4        TV Internet WirelessInternet Airconditioning K...
                               ...                        
49994    TV WirelessInternet Airconditioning Kitchen El...
49995    TV WirelessInternet Airconditioning Kitchen Br...
49996    TV Internet WirelessInternet Airconditioning K...
49997    WirelessInternet Kitchen Petsallowed Buzzer/wi...
49998    Internet WirelessInternet Airconditioning Kitc...
Name: amenities, Length: 49760, dtype: object

### Applying BAG OF WORD(BOW) on amenities feature

In [65]:
from sklearn.feature_extraction.text import CountVectorizer

In [66]:
vectorizer = CountVectorizer()

In [67]:
df = pd.DataFrame(vectorizer.fit_transform(cleaned_train_dataset['amenities']).toarray())

In [68]:

vectorizer.get_feature_names()

['24',
 'accessible',
 'airconditioning',
 'airpurifier',
 'babybath',
 'babymonitor',
 'babysitterrecommendations',
 'bathtowel',
 'bathtub',
 'bathtubwithshowerchair',
 'bbqgrill',
 'beachessentials',
 'beachfront',
 'bedlinens',
 'bodysoap',
 'breakfast',
 'buzzer',
 'cabletv',
 'carbonmonoxidedetector',
 'cat',
 'changingtable',
 'children',
 'cleaningbeforecheckout',
 'coffeemaker',
 'cookingbasics',
 'crib',
 'darkeningshades',
 'disabledparkingspot',
 'dishesandsilverware',
 'dishwasher',
 'dog',
 'doorman',
 'doormanentry',
 'dryer',
 'elevator',
 'elevatorinbuilding',
 'en',
 'essentials',
 'ethernetconnection',
 'evcharger',
 'extrapillowsandblankets',
 'family',
 'fireextinguisher',
 'fireplaceguards',
 'firmmatress',
 'firmmattress',
 'firstaidkit',
 'fixedgrabbarsforshower',
 'flat',
 'flatsmoothpathwaytofrontdoor',
 'freeaccess',
 'freeparkingonpremises',
 'freeparkingonstreet',
 'gameconsole',
 'gardenorbackyard',
 'grab',
 'groundflooraccess',
 'gym',
 'hairdryer',
 'ha

In [69]:
save_model(vectorizer, working_directory+"bow_vectorizer.sav")

In [70]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49755,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
49756,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
49757,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
49758,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [71]:
df.shape

(49760, 145)

In [72]:
cleaned_train_dataset.shape

(49760, 13)

In [73]:
df.to_numpy().shape

(49760, 145)

In [74]:
cleaned_train_dataset.to_numpy().shape

(49760, 13)

In [75]:
cleaned_train_dataset = cleaned_train_dataset.drop(['amenities',],axis = 'columns',inplace = False)

In [76]:
amenities_cleaned_train_dataset = pd.DataFrame(np.concatenate((cleaned_train_dataset.to_numpy(), df.to_numpy()),axis=1))

In [77]:
amenities_cleaned_train_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156
0,Apartment,Entire home/apt,3,1,Real Bed,1,NYC,2,100,1,1,5.01064,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Apartment,Entire home/apt,7,1,Real Bed,1,NYC,6,93,3,3,5.1299,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,Apartment,Entire home/apt,5,1,Real Bed,1,NYC,10,92,1,3,4.97673,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
3,House,Entire home/apt,4,1,Real Bed,1,SF,0,0,2,2,6.62007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
4,Apartment,Entire home/apt,2,1,Real Bed,1,DC,4,40,0,1,4.74493,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49755,Apartment,Private room,1,1,Real Bed,0,NYC,1,100,1,1,4.17439,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
49756,Apartment,Entire home/apt,4,1,Real Bed,1,NYC,0,0,1,1,4.82831,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
49757,Apartment,Entire home/apt,5,1,Real Bed,1,NYC,150,82,2,3,5.0689,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
49758,Apartment,Private room,2,1,Real Bed,1,NYC,19,96,2,1,4.23411,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [78]:
amenities_cleaned_train_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156
0,Apartment,Entire home/apt,3,1,Real Bed,1,NYC,2,100,1,1,5.01064,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Apartment,Entire home/apt,7,1,Real Bed,1,NYC,6,93,3,3,5.1299,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,Apartment,Entire home/apt,5,1,Real Bed,1,NYC,10,92,1,3,4.97673,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
3,House,Entire home/apt,4,1,Real Bed,1,SF,0,0,2,2,6.62007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
4,Apartment,Entire home/apt,2,1,Real Bed,1,DC,4,40,0,1,4.74493,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49755,Apartment,Private room,1,1,Real Bed,0,NYC,1,100,1,1,4.17439,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
49756,Apartment,Entire home/apt,4,1,Real Bed,1,NYC,0,0,1,1,4.82831,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
49757,Apartment,Entire home/apt,5,1,Real Bed,1,NYC,150,82,2,3,5.0689,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1
49758,Apartment,Private room,2,1,Real Bed,1,NYC,19,96,2,1,4.23411,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1


In [79]:
amenities_cleaned_train_dataset.dtypes

0      object
1      object
2      object
3      object
4      object
        ...  
152    object
153    object
154    object
155    object
156    object
Length: 157, dtype: object

In [80]:
amenities_cleaned_train_dataset.columns

RangeIndex(start=0, stop=157, step=1)

### Divide Features and Label

In [81]:
X = amenities_cleaned_train_dataset.iloc[:, amenities_cleaned_train_dataset.columns != 11].values
y = amenities_cleaned_train_dataset.iloc[:, 11].values


### One Hot Encoding of categorical variables

In [82]:
X

array([['Apartment', 'Entire home/apt', 3, ..., 0, 0, 1],
       ['Apartment', 'Entire home/apt', 7, ..., 0, 0, 1],
       ['Apartment', 'Entire home/apt', 5, ..., 0, 1, 1],
       ...,
       ['Apartment', 'Entire home/apt', 5, ..., 0, 1, 1],
       ['Apartment', 'Private room', 2, ..., 0, 1, 1],
       ['House', 'Private room', 2, ..., 0, 0, 1]], dtype=object)

property_type, room_type,bed_type,city are categorical features

In [83]:
print(len(cleaned_train_dataset['property_type'].value_counts()))
cleaned_train_dataset['property_type'].value_counts()

33


Apartment             32955
House                 11083
Condominium            1756
Townhouse              1144
Loft                    831
Other                   389
Guesthouse              324
Bed & Breakfast         316
Bungalow                263
Villa                   119
Dorm                     92
Guest suite              83
Camper/RV                62
In-law                   49
Cabin                    49
Hostel                   48
Timeshare                46
Boutique hotel           44
Boat                     36
Serviced apartment       16
Tent                     12
Castle                    8
Yurt                      7
Vacation home             7
Treehouse                 6
Hut                       5
Chalet                    3
Earth House               2
Tipi                      1
Lighthouse                1
Cave                      1
Train                     1
Casa particular           1
Name: property_type, dtype: int64

In [84]:
print(len(cleaned_train_dataset['room_type'].value_counts()))
cleaned_train_dataset['room_type'].value_counts()

3


Entire home/apt    27811
Private room       20528
Shared room         1421
Name: room_type, dtype: int64

In [85]:
print(len(cleaned_train_dataset['bed_type'].value_counts()))
cleaned_train_dataset['bed_type'].value_counts()

5


Real Bed         48347
Futon              513
Pull-out Sofa      400
Airbed             320
Couch              180
Name: bed_type, dtype: int64

In [86]:
print(len(cleaned_train_dataset['city'].value_counts()))
cleaned_train_dataset['city'].value_counts()

6


NYC        21685
LA         15114
SF          4338
DC          3789
Chicago     2496
Boston      2338
Name: city, dtype: int64

In [87]:
X[:,0]

array(['Apartment', 'Apartment', 'Apartment', ..., 'Apartment',
       'Apartment', 'House'], dtype=object)

In [88]:
X[:,0]

array(['Apartment', 'Apartment', 'Apartment', ..., 'Apartment',
       'Apartment', 'House'], dtype=object)

In [89]:
X[:,1]

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Private room', 'Private room'], dtype=object)

In [90]:
X[:,1]

array(['Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'Private room', 'Private room'], dtype=object)

In [91]:
X[:,4]

array(['Real Bed', 'Real Bed', 'Real Bed', ..., 'Real Bed', 'Real Bed',
       'Real Bed'], dtype=object)

In [92]:
X[:,4]

array(['Real Bed', 'Real Bed', 'Real Bed', ..., 'Real Bed', 'Real Bed',
       'Real Bed'], dtype=object)

In [93]:
X[:,6]

array(['NYC', 'NYC', 'NYC', ..., 'NYC', 'NYC', 'NYC'], dtype=object)

In [94]:
X[:,6]

array(['NYC', 'NYC', 'NYC', ..., 'NYC', 'NYC', 'NYC'], dtype=object)

In [95]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [96]:
X.shape

(49760, 156)

In [97]:
X

array([['Apartment', 'Entire home/apt', 3, ..., 0, 0, 1],
       ['Apartment', 'Entire home/apt', 7, ..., 0, 0, 1],
       ['Apartment', 'Entire home/apt', 5, ..., 0, 1, 1],
       ...,
       ['Apartment', 'Entire home/apt', 5, ..., 0, 1, 1],
       ['Apartment', 'Private room', 2, ..., 0, 1, 1],
       ['House', 'Private room', 2, ..., 0, 0, 1]], dtype=object)

In [98]:
ct = ColumnTransformer([("encoder",
                         OneHotEncoder(handle_unknown='ignore'),
                         [0,1,4,6]
                         )],
                        remainder='passthrough',)

In [99]:
X[0]

array(['Apartment', 'Entire home/apt', 3, 1.0, 'Real Bed', 1, 'NYC', 2,
       100.0, 1.0, 1.0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=object)

In [100]:
X = ct.fit_transform(X)

In [101]:
X[0]

array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3, 1.0, 1, 2, 100.0, 1.0,
       1.0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=object)

In [102]:
X

array([[1.0, 0.0, 0.0, ..., 0, 0, 1],
       [1.0, 0.0, 0.0, ..., 0, 0, 1],
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       [0.0, 0.0, 0.0, ..., 0, 0, 1]], dtype=object)

In [103]:
X.shape

(49760, 199)

In [104]:
save_model(ct, working_directory+'onehotencoder.sav')

Now all features in numberical form.

## Start Training

In [105]:
X.shape

(49760, 199)

In [106]:
y.shape

(49760,)

In [107]:
X

array([[1.0, 0.0, 0.0, ..., 0, 0, 1],
       [1.0, 0.0, 0.0, ..., 0, 0, 1],
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       [1.0, 0.0, 0.0, ..., 0, 1, 1],
       [0.0, 0.0, 0.0, ..., 0, 0, 1]], dtype=object)

In [108]:
X[0]

array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 3, 1.0, 1, 2, 100.0, 1.0,
       1.0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=object)

In [109]:
y

array([5.010635294, 5.129898715, 4.9767337419999995, ..., 5.068904202,
       4.234106505, 3.871201011], dtype=object)

In [110]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

# ML algo

In [111]:
from sklearn import metrics
def evaluate_model(y_test, y_pred):
  print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  #evaluation matrices(MAE, MSE, RMSE)
  print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
  print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))  
  print('R Square Error:', metrics.r2_score(y_test, y_pred))

## Random Forest  Regression

In [112]:
from sklearn.ensemble import RandomForestRegressor        #mse = mean square error
regressor = RandomForestRegressor(n_estimators = 300, random_state = 0)
regressor.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [113]:
y_pred = regressor.predict(x_test)

In [114]:
evaluate_model(y_test, y_pred)


Mean Absolute Error: 0.32956733739290467
Mean Squared Error: 0.19748965220599418
Root Mean Squared Error: 0.4443980785354435
R Square Error: 0.6193473466252062


In [115]:
metrics.explained_variance_score(y_test,y_pred)

0.6194643080392984

In [116]:
save_model(regressor, working_directory+"ml_model.sav")

# Test

In [117]:
Test_Dataset = pd.read_excel(working_directory + 'test.xlsx')


In [118]:
Test_Dataset.head()

,id,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,
0,5979389,Apartment,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",2,1.0,Real Bed,strict,True,NYC,2 bedroom apartment 3 blocks away from the Dek...,NaT,t,t,1.0,2015-05-29,f,NaT,40.705948,-73.915318,Bedroom in Renovated Apartment with Washer/Dryer,Ridgewood,0,NaN,https://a0.muscache.com/im/pictures/b43127ee-b...,11385,1.0,1.0,NaN
1,13488121,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",4,1.0,Real Bed,moderate,True,LA,This 2015 renovation was done with an eye to b...,2015-11-16,t,t,1.0,2015-08-17,f,2017-04-23,34.118408,-118.317929,Peaceful and Quiet Hollywood Hills,Hollywood Hills,24,100.0,https://a0.muscache.com/im/pictures/894d8ca5-7...,90068,1.0,1.0,NaN
2,8121643,Apartment,Private room,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",2,1.0,Real Bed,flexible,False,NYC,Apt features shared living room w/ Apple TV. ...,NaT,t,f,NaN,2012-06-18,f,NaT,40.707888,-74.015211,"Luxury, doorman Building- FIDI",Financial District,0,NaN,NaN,10006,1.0,1.0,NaN
3,16490010,Apartment,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1.0,Real Bed,flexible,True,NYC,Our home is equidistance to the L and G trains...,2016-09-29,t,t,1.0,2011-12-12,f,2017-09-24,40.721388,-73.945642,Cozy room in sunny historic Greenpoint home,Greenpoint,12,98.0,https://a0.muscache.com/im/pictures/6c99615c-4...,11222,1.0,1.0,NaN
4,16274069,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",4,1.0,Real Bed,moderate,True,NYC,JUST 20 MINUTES BY TRAIN TO MANHATTAN - AFFORD...,2015-01-05,t,f,1.0,2014-11-02,f,2017-09-14,40.682556,-73.943259,B's Suite,Bedford-Stuyvesant,13,92.0,https://a0.muscache.com/im/pictures/56275358/c...,11216,2.0,3.0,NaN


In [119]:
Test_Dataset.shape

(24111, 29)

In [120]:
cleaned_test_dataset = Test_Dataset.drop(['id',
                                            'zipcode',
                                            'cancellation_policy',
                                            'description','first_review',
                                            'host_has_profile_pic',
                                            'host_identity_verified',
                                            'host_response_rate',
                                            'host_since',
                                            'instant_bookable',
                                            'last_review', 
                                            'latitude', 
                                            'longitude', 
                                            'name',
                                            'neighbourhood',
                                            'thumbnail_url',
                                            ' '],axis = 'columns',inplace = False)

In [121]:
cleaned_test_dataset.isnull().sum()

property_type              0
room_type                  0
amenities                  0
accommodates               0
bathrooms                 54
bed_type                   0
cleaning_fee               0
city                       0
number_of_reviews          0
review_scores_rating    5421
bedrooms                  29
beds                      40
dtype: int64

In [122]:
cleaned_test_dataset.dtypes

property_type            object
room_type                object
amenities                object
accommodates              int64
bathrooms               float64
bed_type                 object
cleaning_fee               bool
city                     object
number_of_reviews         int64
review_scores_rating    float64
bedrooms                float64
beds                    float64
dtype: object

### Label Encoding

In [123]:
cleaning_fee_label_encoder.classes_

array([False,  True])

In [124]:
cleaned_test_dataset['cleaning_fee'].dtype

dtype('bool')

In [125]:
# Fit in Train section
cleaned_test_dataset['cleaning_fee'] =  cleaning_fee_label_encoder.transform(cleaned_test_dataset['cleaning_fee'])

In [126]:
cleaned_test_dataset['cleaning_fee'].dtype

dtype('int64')

### Handling missing values

In [127]:
cleaned_test_dataset.isnull().sum()

property_type              0
room_type                  0
amenities                  0
accommodates               0
bathrooms                 54
bed_type                   0
cleaning_fee               0
city                       0
number_of_reviews          0
review_scores_rating    5421
bedrooms                  29
beds                      40
dtype: int64

In [128]:
X1_test = cleaned_test_dataset.iloc[:,:].values

In [129]:
X1_test

array([['Apartment', 'Private room',
        '{"Wireless Internet","Air conditioning",Kitchen,Heating,Washer,Dryer,"Smoke detector",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Private living room"}',
        ..., nan, 1.0, 1.0],
       ['Apartment', 'Entire home/apt',
        '{TV,"Cable TV",Internet,"Wireless Internet","Air conditioning",Kitchen,"Free parking on premises",Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Carbon monoxide detector","Safety card","Fire extinguisher",Essentials,Shampoo,"24-hour check-in",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_50",Bathtub,"Room-darkening shades"}',
        ..., 100.0, 1.0, 1.0],
       ['Apartment', 'Private room',
        '{TV,"Cable TV",Internet,"Wireless Internet","Air conditioning",Kitchen,Doorman,Gym,Elevator,Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","First aid kit","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door"

In [130]:
cleaned_test_dataset.shape

(24111, 12)

In [131]:
cleaned_test_dataset['bathrooms']

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
24106    1.0
24107    2.0
24108    1.0
24109    1.0
24110    1.0
Name: bathrooms, Length: 24111, dtype: float64

In [132]:
from sklearn.impute import SimpleImputer
imputer_bt = SimpleImputer(missing_values=np.nan, strategy='mean')

In [133]:
X1_test[:, 4:5]

array([[1.0],
       [1.0],
       [1.0],
       ...,
       [1.0],
       [1.0],
       [1.0]], dtype=object)

In [134]:
X1_test[:, 4:5] = imputer_bt.fit_transform(X1_test[:, 4:5])

In [135]:
save_model(imputer_bt, working_directory+'missingvalues.sav')

In [136]:
imputer_rbb = SimpleImputer(missing_values=np.nan, strategy='mean')

In [137]:
X1_test[:, 9:]

array([[nan, 1.0, 1.0],
       [100.0, 1.0, 1.0],
       [nan, 1.0, 1.0],
       ...,
       [94.0, 2.0, 2.0],
       [nan, 0.0, 2.0],
       [96.0, 1.0, 2.0]], dtype=object)

In [138]:
X1_test[:, 9:] = imputer_rbb.fit_transform(X1_test[:, 9:])

In [139]:
save_model(imputer_rbb, working_directory+'missingvaluesafter.sav')

In [140]:
X1_test[:, 9:]

array([[94.09133226324238, 1.0, 1.0],
       [100.0, 1.0, 1.0],
       [94.09133226324238, 1.0, 1.0],
       ...,
       [94.0, 2.0, 2.0],
       [94.09133226324238, 0.0, 2.0],
       [96.0, 1.0, 2.0]], dtype=object)

In [141]:
X1_test

array([['Apartment', 'Private room',
        '{"Wireless Internet","Air conditioning",Kitchen,Heating,Washer,Dryer,"Smoke detector",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Private living room"}',
        ..., 94.09133226324238, 1.0, 1.0],
       ['Apartment', 'Entire home/apt',
        '{TV,"Cable TV",Internet,"Wireless Internet","Air conditioning",Kitchen,"Free parking on premises",Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","Carbon monoxide detector","Safety card","Fire extinguisher",Essentials,Shampoo,"24-hour check-in",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","translation missing: en.hosting_amenity_50",Bathtub,"Room-darkening shades"}',
        ..., 100.0, 1.0, 1.0],
       ['Apartment', 'Private room',
        '{TV,"Cable TV",Internet,"Wireless Internet","Air conditioning",Kitchen,Doorman,Gym,Elevator,Heating,"Family/kid friendly",Washer,Dryer,"Smoke detector","First aid kit","Fire extinguisher",Essentials,Shampoo,"Lock on

### Converting amenties into BOW

In [142]:
# Fitted in Train section
df = pd.DataFrame(vectorizer.transform(X1_test[:,2]).toarray())

In [143]:
X1_test_df = pd.DataFrame(X1_test)

In [144]:
X1_test_df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
dtype: int64

In [145]:
df.shape

(24111, 145)

In [146]:
X1_test_df.shape

(24111, 12)

In [147]:
X1_test_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,Apartment,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",2,1,Real Bed,1,NYC,0,94.0913,1,1
1,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",4,1,Real Bed,1,LA,24,100,1,1
2,Apartment,Private room,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",2,1,Real Bed,0,NYC,0,94.0913,1,1
3,Apartment,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1,Real Bed,1,NYC,12,98,1,1
4,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",4,1,Real Bed,1,NYC,13,92,2,3


In [148]:
X1_test_df.columns

RangeIndex(start=0, stop=12, step=1)

In [149]:
X1_test_df.drop([2],axis = 'columns',inplace = True)

In [150]:
amenities_cleaned_test_dataset = pd.DataFrame(np.concatenate((X1_test_df.to_numpy(), df.to_numpy()),axis=1))

In [151]:
amenities_cleaned_test_dataset.columns

RangeIndex(start=0, stop=156, step=1)

In [152]:
amenities_cleaned_test_dataset.shape

(24111, 156)

### OneHotEncoding of categorical columns

In [153]:
X_test = amenities_cleaned_test_dataset.iloc[:, :].values

In [154]:
X_test.shape

(24111, 156)

In [155]:
X_test

array([['Apartment', 'Private room', 2, ..., 0, 0, 0],
       ['Apartment', 'Entire home/apt', 4, ..., 0, 0, 0],
       ['Apartment', 'Private room', 2, ..., 0, 0, 0],
       ...,
       ['Apartment', 'Entire home/apt', 5, ..., 0, 0, 0],
       ['Apartment', 'Entire home/apt', 2, ..., 0, 0, 0],
       ['Boat', 'Entire home/apt', 4, ..., 0, 0, 0]], dtype=object)

In [156]:
print(len(cleaned_test_dataset['property_type'].value_counts()))
cleaned_test_dataset['property_type'].value_counts()

32


Apartment             15890
House                  5373
Condominium             898
Townhouse               546
Loft                    410
Other                   215
Guesthouse              174
Bed & Breakfast         142
Bungalow                102
Villa                    59
Dorm                     43
Guest suite              40
Timeshare                31
Camper/RV                31
Boat                     29
Boutique hotel           25
Cabin                    23
Hostel                   22
In-law                   22
Tent                      6
Castle                    5
Serviced apartment        5
Vacation home             4
Hut                       3
Chalet                    3
Earth House               2
Tipi                      2
Yurt                      2
Train                     1
Treehouse                 1
Island                    1
Cave                      1
Name: property_type, dtype: int64

In [157]:
print(len(cleaned_test_dataset['room_type'].value_counts()))
cleaned_test_dataset['room_type'].value_counts()

3


Entire home/apt    13405
Private room        9980
Shared room          726
Name: room_type, dtype: int64

In [158]:
print(len(cleaned_test_dataset['bed_type'].value_counts()))
cleaned_test_dataset['bed_type'].value_counts()

5


Real Bed         23452
Futon              235
Pull-out Sofa      182
Airbed             154
Couch               88
Name: bed_type, dtype: int64

In [159]:
print(len(cleaned_test_dataset['city'].value_counts()))
cleaned_test_dataset['city'].value_counts()

6


NYC        10545
LA          7262
SF          2077
DC          1884
Chicago     1221
Boston      1122
Name: city, dtype: int64

In [160]:
X_test[:,0]

array(['Apartment', 'Apartment', 'Apartment', ..., 'Apartment',
       'Apartment', 'Boat'], dtype=object)

In [161]:
X_test[:,1]

array(['Private room', 'Entire home/apt', 'Private room', ...,
       'Entire home/apt', 'Entire home/apt', 'Entire home/apt'],
      dtype=object)

In [162]:
X_test[:,4]

array(['Real Bed', 'Real Bed', 'Real Bed', ..., 'Real Bed', 'Real Bed',
       'Real Bed'], dtype=object)

In [163]:
X_test[:,6]

array(['NYC', 'LA', 'NYC', ..., 'NYC', 'NYC', 'LA'], dtype=object)

In [164]:
X_test[0]

array(['Apartment', 'Private room', 2, 1.0, 'Real Bed', 1, 'NYC', 0,
       94.09133226324238, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=object)

In [165]:
X2_test = ct.transform(X_test)

In [166]:
X2_test[0]

array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 2, 1.0, 1, 0,
       94.09133226324238, 1.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=object)

In [167]:
X2_test.shape

(24111, 199)

### Prediction

In [168]:
y_test_pred = regressor.predict(X2_test)

In [169]:
y_test_pred.shape

(24111,)

In [170]:
y_test_pred

array([4.42901245, 4.83223054, 4.65446078, ..., 5.32859056, 4.57469913,
       4.89682983])

In [171]:
Test_Dataset['id'].shape

(24111,)

In [172]:
Test_Dataset['id']

0         5979389
1        13488121
2         8121643
3        16490010
4        16274069
           ...   
24106    14549287
24107    13281809
24108    18688039
24109    17045948
24110     3534845
Name: id, Length: 24111, dtype: int64

In [173]:
sub_dict = {"id":Test_Dataset['id'], "log_price": y_test_pred}

In [174]:
pd.DataFrame(sub_dict).to_csv(working_directory+'submission.csv', index=False)